In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config
from context import Context
from contracts.farm_contract import FarmContract
from contracts.fees_collector_contract import FeesCollectorContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from contracts.pair_contract import PairContract
from contracts.router_contract import RouterContract
from contracts.builtin_contracts import ESDTContract
from utils.utils_generic import log_warning, log_step_pass, log_step_fail

context = Context()
fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]

log_step_pass(f"Fees collector contract: {fees_collector_contract.address}")
log_step_pass(f"Energy contract: {energy_contract.address}")
log_step_pass(f"Router contract: {router_contract.address}")

ush_farm_contract = FarmContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqvymupfsf40v7xgtkcgtzdnstd9x9xfgzkp2s8wnwpz")
ush_pair_contract = PairContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqs8r2jhfymgle49dqx42xyypx6r2smt602jps2kcn8f")

esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)

### Farm rewards distribution

In [ ]:
import json

rewards_config_file = "emission_rewards_25_03_03.json"
log_warning(f"Reading rewards config from {rewards_config_file}")

# load the rewards json file
with open(rewards_config_file) as f:
    emission_rewards = json.load(f)

In [ ]:
from tools.runners.farm_runner import get_farm_addresses_from_chain
from utils.utils_generic import get_continue_confirmation
from utils.utils_chain import get_account_key, string_to_hex

# set the rewards
farm_addresses = get_farm_addresses_from_chain("v2")
log_step_pass(f"Retrieved {len(farm_addresses)} farm addresses.")
if get_continue_confirmation():
    for farm_address in farm_addresses:
        try:
            farm_contract: FarmContract
            farm_contract = FarmContract.load_contract_by_address(farm_address)
            log_step_pass(f"Setting rewards for farm {farm_contract.farmToken} {farm_contract.address}")
        except AttributeError as e:
            log_step_fail(f"Failed to load farm contract {farm_address}: {e}")
            if not get_continue_confirmation():
                break
            continue
        
        if farm_contract.farmToken not in emission_rewards:
            current_rpb = farm_contract.get_per_block_reward_amount(context.network_provider.proxy)
            log_step_fail(f"Farm token {farm_contract.farmToken} not found in rewards config. \nCurrent rewards: {current_rpb:,}. \nSkipping.")
            continue

        new_rewards = emission_rewards[farm_contract.farmToken] * 10**18
        log_warning(f"New rewards: {new_rewards:,}")

        if not new_rewards:
            new_rewards = 1     # can't set 0 rpb
            # check if rewards producing is stopped, otherwise offer to stop it
            rewards_enabled = get_account_key(farm_address, string_to_hex("produce_rewards_enabled"), context.network_provider.proxy)
            if rewards_enabled == "01":
                log_warning(f"Produce rewards enabled for farm {farm_contract.farmToken}. Want to stop?")
                if get_continue_confirmation():
                    tx_hash = farm_contract.end_produce_rewards(context.deployer_account, context.network_provider.proxy)
                    if not context.network_provider.check_simple_tx_status(tx_hash, f"stop rewards for: {farm_address}"):
                        if not get_continue_confirmation():
                            break
                
        # if not get_continue_confirmation():
        #      break
        current_rpb = farm_contract.get_per_block_reward_amount(context.network_provider.proxy)
        if current_rpb == new_rewards:
            log_warning(f"Rewards for farm {farm_contract.farmToken} are already set to {new_rewards:,}. Skipping.")
            continue

        tx_hash = farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, new_rewards)
        if not context.network_provider.check_simple_tx_status(tx_hash, f"set rewards per block for: {farm_address}"):
                if not get_continue_confirmation():
                    break

log_step_pass("All done.")

Fees collector rewards distribution

In [ ]:
new_rewards = emission_rewards["FEESCOLLECTOR"] * 10**18
log_warning(f"New fees collector rewards: {new_rewards:,}")
tx_hash = fees_collector_contract.set_locked_tokens_per_block(context.deployer_account, context.network_provider.proxy, new_rewards)
context.network_provider.check_simple_tx_status(tx_hash, f"set fees collector rewards per block")

### USH farm setup

In [ ]:
tx_hash = ush_pair_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, ush_farm_contract.address)
tx_hash = energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, ush_farm_contract.address)
tx_hash = energy_contract.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [ush_farm_contract.address])

In [ ]:
tx_hash = ush_farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 14000 * 10**18)

In [ ]:
tx_hash = ush_farm_contract.resume(context.deployer_account, context.network_provider.proxy)

In [ ]:
tx_hash = ush_farm_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)

### Pairs buyback and burn setup

In [ ]:
pair_addresses = [
    "erd1qqqqqqqqqqqqqpgq3ugz94td483jk7rypkd2vknta8y3q26u2jpsqrxvhx",
    "erd1qqqqqqqqqqqqqpgqwlfdr9heygm0t639v94ktp8hptu4qhra2jps8gtqjh",
    "erd1qqqqqqqqqqqqqpgqlt8sksgnhk98pm2chnjwhz5cat7s5wy72jpsgdrmac",
    "erd1qqqqqqqqqqqqqpgqh6tzuhfzl7myeq0s9eavzjh5ax9h77kt2jpsnw0yeu",
    "erd1qqqqqqqqqqqqqpgqs8r2jhfymgle49dqx42xyypx6r2smt602jps2kcn8f"
    ]

context.deployer_account.sync_nonce(context.network_provider.proxy)

mex_ush_pair = PairContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqyghvzwv9qyzq95vwhvk4tafg945r8hcf2jps0x7me7")

# mex burn role
tx_hash = esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy, ["MEX-455c57", mex_ush_pair.address, "ESDTRoleLocalBurn"])
if not context.network_provider.check_complex_tx_status(tx_hash, f"set burn role for MEX: {farm_address}"):
    if not get_continue_confirmation():
        raise Exception("Failed to set burn role for MEX")

mex_ush_pair.set_fees_percents(context.deployer_account, context.network_provider.proxy,
                                [300, 100])
    
mex_ush_pair.set_fee_on_via_router(context.deployer_account, context.network_provider.proxy, router_contract, 
                            [
                                config.ZERO_CONTRACT_ADDRESS,
                                mex_ush_pair.secondToken
                            ])

fees_collector_contract.add_known_contracts(context.deployer_account, context.network_provider.proxy,
                                            [mex_ush_pair.address])

mex_ush_pair.add_fees_collector(context.deployer_account, context.network_provider.proxy,
                                [fees_collector_contract.address, 50000])

input("Starting to setup pairs. Press Enter to continue...")

for pair_address in pair_addresses:
    pair_contract = PairContract.load_contract_by_address(pair_address)    # operating pair

    print(f"Setting up pair {pair_contract.lpToken}: {pair_address}")

    # whitelist in egldmex pair for swap no fees
    mex_ush_pair.whitelist_contract(context.deployer_account, context.network_provider.proxy, pair_contract.address)

    # set where to swap and what to do with the fees
    pair_contract.add_trusted_swap_pair(context.deployer_account, context.network_provider.proxy,
                                        [
                                            mex_ush_pair.address,
                                            mex_ush_pair.firstToken,
                                            mex_ush_pair.secondToken
                                        ])
    
    pair_contract.set_fees_percents(context.deployer_account, context.network_provider.proxy,
                                [300, 100])
    
    pair_contract.set_fee_on_via_router(context.deployer_account, context.network_provider.proxy, router_contract, 
                                [
                                    config.ZERO_CONTRACT_ADDRESS,
                                    mex_ush_pair.secondToken
                                ])
    
    fees_collector_contract.add_known_contracts(context.deployer_account, context.network_provider.proxy,
                                                [
                                                    pair_contract.address
                                                ])
    pair_contract.add_fees_collector(context.deployer_account, context.network_provider.proxy,
                                    [fees_collector_contract.address, 50000])
    
    input("Press Enter to continue...")